In [6]:
import os, glob
import pandas as pd

import time

import pprint
import os
import sys
import html
import datetime
import pandas as pd
from bs4 import BeautifulSoup
import re
from datetime import datetime

In [13]:
def get_xml_data(filename):

    home_dir = "/content/drive/MyDrive/dataset_patent/2019"
    file_path =home_dir+filename

   
    df_pos = pd.DataFrame(columns=['publication_number', 'patent_title', 'appl_type', 
                           'positive_text'])  # positive_text,   
    df_neg = pd.DataFrame(columns=['publication_number', 'patent_title', 'appl_type', 
                           'negative_text']) # negative_text
    df_neut = pd.DataFrame(columns=['publication_number', 'patent_title', 'appl_type', 
                           'neutral_text'])  # neutral_text 
   
    total_positive_samples = 0
    total_negative_samples = 0
    total_neutral_samples = 0

    total_publications = 0
    xml_text = html.unescape(open(filename, 'r').read())
    for patent in xml_text.split("<?xml version=\"1.0\" encoding=\"UTF-8\"?>"):

        if patent is None or patent == "":
            continue
        patent_text = patent
    
        bs = BeautifulSoup(patent, "lxml")

        fwu_advantages = bs.find('heading',text='Advantageous Effects of Invention')
        fwu_disadvantages = bs.find('heading',text='Technical Problem')
        fwu_solutions = bs.find('heading',text='Solution to Problem')

        #Advantageous Effects of Invention --- for positive samples
        #Technical Problem --- for negative samples
        #Solution to Problem --- for neutral samples, also the paragraphs (2/3) which are just above the 'Advantageous Effects of Invention' section can be used for neural samples 

    #### Search for application or grant
        application = bs.find('us-patent-application')
        if application is None: # If no application, search for grant
            application = bs.find('us-patent-grant')
        
        title = "None"
            
    #### Search for its title
        try:
            title = application.find('invention-title').text
        except Exception as e:          
            #print("no title found")
            title = ""

    #### Search for publication number
        try:
            publication_kind = application.find('publication-reference').find('kind').text
        except Exception as e:
            publication_kind = ""
        
        try:
            publication_num = application['file'].split("-")[0]
        except Exception as e:
            publication_num = ""
        
        publication_num = publication_num+publication_kind
    
        try:
            application_type = application.find('application-reference')['appl-type']
        
        except Exception as e:
            application_type =''
            
        if publication_num:
            total_publications +=1

#### For positive labels 

        if fwu_advantages:
            total_positive_samples +=1
        
            text = patent_text.splitlines()
            adv = []
            pos_text=""

            for i in text:
                start = text.index(i)
                iteration = 1
                if ('>Advantageous Effects of Invention<'in i and iteration==1):
                    for j in range(20):
                        if '<p' in text[start+1]:
                            pos_text = text[start+1]
                            adv.append(text[start+1])
                            start = start+1
                        else:
                            continue
                    df_pos = df_pos.append({'publication_number':publication_num, 'patent_title': title,
                                'appl_type':application_type,
                               'positive_text':adv},ignore_index=True)
                
                    iteration=0

#### For negative labels
      
        if fwu_disadvantages:
            total_negative_samples +=1
        
            text = patent_text.splitlines()
            disadv = []
            neg_text=""

            for i in text:
                start = text.index(i)
                iteration = 1
                if ('>Technical Problem<'in i and iteration==1):
                    for j in range(20):
                        if '<p' in text[start+1]:
                            neg_text = text[start+1]
                            disadv.append(text[start+1])
                            start = start+1
                        else:
                            continue
                    df_neg = df_neg.append({'publication_number':publication_num, 'patent_title': title,
                                'appl_type':application_type,
                               'negative_text':disadv},ignore_index=True)
                
                    iteration=0
      
#### For neutral labels 

        if fwu_solutions:
            total_neutral_samples +=1
        
            text = patent_text.splitlines()
            neut = []
            neut_text=""

            for i in text:
                start = text.index(i)
                iteration = 1
                if ('>Solution to Problem<'in i and iteration==1):
                    for j in range(20):
                        if '<p' in text[start+1]:
                            neut_text = text[start+1]
                            neut.append(text[start+1])
                            start = start+1
                        else:
                            continue
                    df_neut = df_neut.append({'publication_number':publication_num, 'patent_title': title,
                                'appl_type':application_type,
                               'neutral_text':neut},ignore_index=True)
                
                    iteration=0                                 

        
    file = filename
    name = file.split('.')
    name_pos =name[0]+'_pos.csv'
    name_neg =name[0]+'_neg.csv'
    name_neut =name[0]+'_neut.csv'
  
    df_pos.to_csv(name_pos)  # for each week, there will be a seperate csv file created and later you can merge them
    df_neg.to_csv(name_neg)
    df_neut.to_csv(name_neut)

    print('total publications in',filename,total_publications )
    print('total positive samples in ', filename,total_positive_samples )
    print('total publications in',filename,total_publications )
    print('total negative samples in ', filename,total_negative_samples )
    print('total publications in',filename,total_publications )
    print('total neutral samples in ', filename,total_neutral_samples )

    
    print(df_pos.shape)
    print("------------")
    print(df_neg.shape)
    print("------------")
    print(df_neut.shape)
    print("------------")

    return filename, total_publications, total_positive_samples, total_negative_samples, total_neutral_samples, df_pos, df_neg, df_neut   

In [14]:
df_stats_pos = pd.DataFrame(columns=['filename', 'total_publications', 'total_positive_samples'])
df_stats_neg = pd.DataFrame(columns=['filename', 'total_publications', 'total_negative_samples'])
df_stats_neut = pd.DataFrame(columns=['filename', 'total_publications', 'total_neutral_samples'])

In [15]:
df_apps_pos = pd.DataFrame(columns=['publication_number', 'patent_title', 'appl_type', 'positive_text'])
df_apps_neg = pd.DataFrame(columns=['publication_number', 'patent_title', 'appl_type', 'negative_text'])
df_apps_neut = pd.DataFrame(columns=['publication_number', 'patent_title', 'appl_type', 'neutral_text'])

In [16]:
start = datetime.now()

In [17]:
path = "/content/drive/MyDrive/dataset_patent/2019"
all_files = glob.glob(os.path.join(path, "ipg*.xml"))
for filename in all_files:
    result = get_xml_data(filename)
    df_stats_pos = df_stats_pos.append({'filename':result[0][-13:-4], 'total_publications':result[1], 'total_positive_samples': result[2]},ignore_index=True)
    df_stats_neg = df_stats_neg.append({'filename':result[0][-13:-4], 'total_publications':result[1], 'total_negative_samples': result[3]},ignore_index=True)
    df_stats_neut = df_stats_neut.append({'filename':result[0][-13:-4], 'total_publications':result[1], 'total_neutral_samples': result[4]},ignore_index=True)

    df_apps_pos = df_apps_pos.append(result[5])
    df_apps_neg = df_apps_neg.append(result[6])
    df_apps_neut = df_apps_neut.append(result[7])

total publications in /content/drive/MyDrive/dataset_patent/2019/ipg190101.xml 7126
total positive samples in  /content/drive/MyDrive/dataset_patent/2019/ipg190101.xml 162
total publications in /content/drive/MyDrive/dataset_patent/2019/ipg190101.xml 7126
total negative samples in  /content/drive/MyDrive/dataset_patent/2019/ipg190101.xml 257
total publications in /content/drive/MyDrive/dataset_patent/2019/ipg190101.xml 7126
total neutral samples in  /content/drive/MyDrive/dataset_patent/2019/ipg190101.xml 204
(162, 4)
------------
(257, 4)
------------
(204, 4)
------------
total publications in /content/drive/MyDrive/dataset_patent/2019/ipg190212.xml 5689
total positive samples in  /content/drive/MyDrive/dataset_patent/2019/ipg190212.xml 125
total publications in /content/drive/MyDrive/dataset_patent/2019/ipg190212.xml 5689
total negative samples in  /content/drive/MyDrive/dataset_patent/2019/ipg190212.xml 209
total publications in /content/drive/MyDrive/dataset_patent/2019/ipg190212.

In [18]:
df_stats_pos.to_csv(path+'/20'+result[0][-10:-8]+'_pos_stats.csv')
df_stats_neg.to_csv(path+'/20'+result[0][-10:-8]+'_neg_stats.csv')
df_stats_neut.to_csv(path+'/20'+result[0][-10:-8]+'_neut_stats.csv')

In [22]:
df_apps_pos.to_csv(path+'/20'+result[0][-10:-8]+'_pos_apps.csv')
df_apps_neg.to_csv(path+'/20'+result[0][-10:-8]+'_neg_apps.csv')
df_apps_neut.to_csv(path+'/20'+result[0][-10:-8]+'_neut_apps.csv')

In [20]:
finish  = datetime.now()

In [21]:
print(finish-start)

0:26:28.039698
